<a href="https://colab.research.google.com/github/Codyd2k1/Lung-Health-Diagnonsis-Tool/blob/main/Lung_Health_Diagnosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lung Health Diagnosis Tool**

**Importing Data from Competition**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
import matplotlib.pyplot as plt
import os
import shutil
import keras_preprocessing
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import keras
from PIL import Image
from tempfile import TemporaryFile

In [ ]:
pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 7.0MB/s 


In [ ]:
import kerastuner as kt

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

ValueError: ignored

In [ ]:
trainingData = pd.read_csv("/content/drive/MyDrive/AS5_Files/assignment5_training_data_metadata.csv")
testingData = pd.read_csv("/content/drive/MyDrive/AS5_Files/assignment5_test_data_metadata.csv")

In [ ]:
os.rename(r'/content/drive/MyDrive/AS5_Files/test/all_classes/IM-0021-0001.jpeg',r('/content/drive/MyDrive/AS5_Files/test/all_classes/' + ))

**Cleaning & Data Preparation**

In [ ]:
#rename the files in testing data to their id's.
testDataPath = "/content/drive/MyDrive/AS5_Files/test/all_classes/"
for i in range(0,testingData.shape[0]):
  row = testingData.iloc[i]
  tmPath = testDataPath + row.image_name
  os.rename(r"" + tmPath,r"" + testDataPath + str(row.id)+".jpeg")

Because the file Names of the Test Data set identify whether they're normal, bacteria, or virus, i will convert this set into my validation set, to see my actual leaderboard score in this notebook.

In [ ]:
testingData.head()

,id,image_name
0,5309,IM-0021-0001.jpeg
1,5310,IM-0019-0001.jpeg
2,5311,IM-0017-0001.jpeg
3,5312,IM-0016-0001.jpeg
4,5313,IM-0015-0001.jpeg


In [ ]:
testingData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 624 entries, 0 to 623
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          624 non-null    int64 
 1   image_name  624 non-null    object
dtypes: int64(1), object(1)
memory usage: 9.9+ KB


In [ ]:
imgName = "appbacterialfa"
if "bacteria" in imgName:
  print("yes")

yes


In [ ]:
for i in range(0,testingData.shape[0]):
  row = testingData.iloc[i]
  imgName = str(row.image_name)
  if "virus" in imgName:
    shutil.move(("/content/drive/MyDrive/AS5_Files/test/all_classes/" + str(row.id) + ".jpeg"), ('/content/drive/MyDrive/AS5_Files/test/Virus/' + str(row.id) + ".jpeg"))
  #elif "NORMAL" in imgName:
  #  shutil.move(("/content/drive/MyDrive/AS5_Files/test/all_classes/" + str(row.id) + ".jpeg"), ('/content/drive/MyDrive/AS5_Files/test/Normal/' + str(row.id) + ".jpeg"))
  #elif "IM" in imgName:
  #  shutil.move(("/content/drive/MyDrive/AS5_Files/test/all_classes/" + str(row.id) + ".jpeg"), ('/content/drive/MyDrive/AS5_Files/test/Normal/' + str(row.id) + ".jpeg"))    

Normalize Image Data:

In [ ]:
trainingPath = "/content/drive/MyDrive/AS5_Files/hyperTraining/train/"

In [ ]:
trainingData['type'].isnull().sum()

0

In [ ]:
trainingData['type'] = trainingData['type'].fillna(1)

In [ ]:
trainingData.shape

(5286, 5)

Splitting Training Data into "Training" and "Validation" sets:

In [ ]:
originalData = trainingData
originalData = originalData.sample(frac = 1, random_state= 42) 

In [ ]:
trainingData.shape

(5286, 5)

In [ ]:
trainingSet = originalData.iloc[:4758,:]
validationSet = originalData.iloc[4759:,:]

In [ ]:
trainingSet = trainingData

Now that we have the data for our validation set and our training set, we want to move the data associated with each set into each folder (training and Validation).

In [ ]:
def move_items_to_proper_folder(dataset, setName):
  for i in range(0,dataset.shape[0]):
    item = dataset.iloc[i]
    shutil.move(("/content/drive/MyDrive/AS5_Files/train/" + item.image_name), ('/content/drive/MyDrive/AS5_Files/' + setName + "/" + item.image_name))

In [ ]:
move_items_to_proper_folder(validationSet, "Validation")

Now we have the data split into proper directories, we need to split the directories into pnemonia/normal class folders.

In [ ]:
def move_items_to_proper_subFolder(dataSet, setName):
  if setName == "Validation":
    tmpPath = "/content/drive/MyDrive/AS5_Files/Validation"
  elif setName == "train":
    tmpPath = "/content/drive/MyDrive/AS5_Files/hyperTraining/train"
  for i in range(0, dataSet.shape[0]):
    item = dataSet.iloc[i]
    if item.type == "bacteria":
      shutil.move((tmpPath + "/" + item.image_name), (tmpPath + "/Bacteria/" + item.image_name))
    elif item.type == "Virus":
      shutil.move((tmpPath + "/" + item.image_name), (tmpPath + "/Virus/" + item.image_name))
    elif item.label == "Normal":
      shutil.move((tmpPath + "/" + item.image_name), (tmpPath + "/Normal/" + item.image_name))
    else:
      continue

In [ ]:
move_items_to_proper_subFolder(validationSet, "Validation")

In [ ]:
move_items_to_proper_subFolder(trainingSet, "train")

**Building & Training Neural Network**

Preparing for CNN Model with Keras:

In [ ]:
training_Path = "/content/drive/MyDrive/AS5_Files/hyperTraining/train"
validation_Path = "/content/drive/MyDrive/AS5_Files/test"
testing_Path = "/content/drive/MyDrive/AS5_Files/test"
small_training_path = "/content/drive/MyDrive/AS5_Files/small_training_batch"

    Set up IMG Generator Batches

In [ ]:
training_Batches = ImageDataGenerator(rescale =1. / 255,
                                      rotation_range=15,
                                      shear_range=0.2,
                                      zoom_range=0.2).flow_from_directory(training_Path, target_size = (192,192), classes=['Normal', 'Virus','Bacteria'], batch_size = 50, color_mode = "rgb", shuffle= True, seed=42)
validation_Batches = ImageDataGenerator().flow_from_directory(validation_Path, target_size = (192,192), classes=['Normal', 'Virus','Bacteria'], batch_size = 50, color_mode = "rgb", seed=42)

Found 5284 images belonging to 3 classes.
Found 624 images belonging to 3 classes.


In [ ]:
testing_Batches = ImageDataGenerator(rescale =1. / 255).flow_from_directory(testing_Path, target_size = (192,192), batch_size = 50, color_mode = "rgb", shuffle = False, seed = 42)

Found 624 images belonging to 4 classes.


In [ ]:
small_training_batch = ImageDataGenerator().flow_from_directory(small_training_path, target_size = (192,192), classes=['Bacteria','Normal', 'Virus'], batch_size = 3, color_mode = "rgb", seed = 42)

Found 9 images belonging to 3 classes.


In [ ]:

def model_builder(hp):
  model = keras.Sequential()
  model.add(Conv2D(32,(3,3), input_shape = (192,192,3), activation='relu'))#, activation='relu'))
  model.add(
      Conv2D(
      filters = hp.Int('convfilter1val',
                         min_value = 64, max_value = 128, step = 32),
                         kernel_size = (3,3),
                         activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(
      Dropout(rate = hp.Float(
          'dropoutVal1',
          min_value = 0.0,
          max_value = 0.5,
          default = 0.25,
          step = 0.05
      )    
      ))
  model.add(Flatten())
  model.add(
      Dense(
      units = hp.Int(
          'unitVal1',
          min_value = 32,
          max_value = 512,
          step = 32,
          default = 64
      ),activation='relu'))
  model.add(
      Dropout(rate = hp.Float(
          'dropoutVal2',
          min_value = 0.0,
          max_value = 0.5,
          default = 0.25,
          step = 0.05
      )    
      ))
  model.add(Dense(3,activation="softmax"))
  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  #hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  #model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  #model.add(keras.layers.Dense(3))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss= 'categorical_crossentropy',
                metrics=['accuracy'])

  return model



In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     seed = 42,
                     directory="/content/drive/MyDrive/AS5_Files/",
                     project_name= "untitled_project"
                    )

In [ ]:
eStopper = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose=1, patience=5)

In [ ]:
tuner.search(training_Batches, steps_per_epoch = 12, validation_data = validation_Batches, callbacks=[eStopper], epochs = 10)

Trial 23 Complete [00h 05m 08s]
val_accuracy: 0.6153846383094788

Best val_accuracy So Far: 0.6634615659713745
Total elapsed time: 01h 17m 32s

Search: Running Trial #24

Hyperparameter    |Value             |Best Value So Far 
convfilter1val    |64                |64                
dropoutVal1       |0.45              |0.5               
unitVal1          |320               |480               
dropoutVal2       |0.25              |0.45              
learning_rate     |0.001             |0.0001            
tuner/epochs      |4                 |4                 
tuner/initial_e...|0                 |0                 
tuner/bracket     |1                 |1                 
tuner/round       |0                 |0                 

Epoch 1/4
12/12 [==============================] - 31s 3s/step - loss: 18.9885 - accuracy: 0.4589 - val_loss: 87.9731 - val_accuracy: 0.3910
Epoch 2/4
12/12 [==============================] - 25s 2s/step - loss: 1.0330 - accuracy: 0.5233 - val_loss: 112.0136

NotFoundError: ignored

In [ ]:
tuner.get_best_models()[0].save('model1')

NotFoundError: ignored

Building CNN

In [ ]:
bestParams = tuner.get_best_hyperparameters()[0]
newModel = tuner.hypermodel.build(bestParams)

In [ ]:
newModel = tuner.get_best_models(num_models=1)[0]

In [ ]:
training_Batches.reset()
validation_Batches.reset()

In [ ]:
hist = newModel.fit(training_Batches, validation_data= validation_Batches, epochs = 20, steps_per_epoch= 20)

Epoch 1/20
20/20 [==============================] - 771s 38s/step - loss: 0.3489 - accuracy: 0.8419 - val_loss: 77.5154 - val_accuracy: 0.7605
Epoch 2/20
20/20 [==============================] - 404s 20s/step - loss: 0.3152 - accuracy: 0.8744 - val_loss: 79.3963 - val_accuracy: 0.7643
Epoch 3/20
20/20 [==============================] - 321s 16s/step - loss: 0.2906 - accuracy: 0.8818 - val_loss: 89.4473 - val_accuracy: 0.7624
Epoch 4/20
20/20 [==============================] - 263s 13s/step - loss: 0.3019 - accuracy: 0.8785 - val_loss: 69.5940 - val_accuracy: 0.7662
Epoch 5/20
20/20 [==============================] - 192s 10s/step - loss: 0.2800 - accuracy: 0.9025 - val_loss: 92.7218 - val_accuracy: 0.7700
Epoch 6/20
20/20 [==============================] - 164s 8s/step - loss: 0.2431 - accuracy: 0.9155 - val_loss: 76.1463 - val_accuracy: 0.7662
Epoch 7/20
20/20 [==============================] - 128s 6s/step - loss: 0.2666 - accuracy: 0.8932 - val_loss: 96.3263 - val_accuracy: 0.7510
E

In [ ]:
epoch_val_accs = hist.history['val_accuracy']
best_epoch_val = epoch_val_accs.index(max(epoch_val_accs)) + 1
print(best_epoch_val)

17


In [ ]:
newModelV1 = tuner.hypermodel.build(bestParams)

In [ ]:
training_Batches.reset()
testing_Batches.reset()
validation_Batches.reset()

In [ ]:
newModelV1.fit(training_Batches, validation_data= validation_Batches, epochs = best_epoch_val, steps_per_epoch= 20 )

Epoch 1/17
20/20 [==============================] - 29s 1s/step - loss: 2.0572 - accuracy: 0.4306 - val_loss: 176.8689 - val_accuracy: 0.6692
Epoch 2/17
20/20 [==============================] - 22s 1s/step - loss: 0.9077 - accuracy: 0.6353 - val_loss: 48.0224 - val_accuracy: 0.6882
Epoch 3/17
20/20 [==============================] - 26s 1s/step - loss: 0.6755 - accuracy: 0.7038 - val_loss: 60.6606 - val_accuracy: 0.7414
Epoch 4/17
20/20 [==============================] - 19s 979ms/step - loss: 0.5538 - accuracy: 0.7811 - val_loss: 61.4166 - val_accuracy: 0.7814
Epoch 5/17
20/20 [==============================] - 18s 960ms/step - loss: 0.5648 - accuracy: 0.7477 - val_loss: 55.9678 - val_accuracy: 0.7681
Epoch 6/17
20/20 [==============================] - 19s 967ms/step - loss: 0.5112 - accuracy: 0.7952 - val_loss: 69.2503 - val_accuracy: 0.7681
Epoch 7/17
20/20 [==============================] - 20s 1s/step - loss: 0.4441 - accuracy: 0.8036 - val_loss: 58.2501 - val_accuracy: 0.7871
Epo

In [ ]:
newPred = newModel.predict(testing_Batches)

After going through with a baseline model of just one Conv2d Layer, I now will up the layers to 4 convolution layers and 2 connected layers.

In [ ]:
###
#model = Sequential([
#                    #conv2d layer
#                    Conv2D(32,(3,3), activation = "relu", input_shape = (400,400,1),),
#                    Flatten(),
#                    Dense(3, activation = "softmax")
#])
###

In [ ]:
model = Sequential()

  layer 1

In [ ]:
model.add(Conv2D(32,(3,3), input_shape = (192,192,3), activation='relu'))#, activation='relu'))
model.add(Conv2D(64,(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(.25))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(.5))
model.add(Dense(3,activation="softmax"))




next 2 layers

In [ ]:
#
#n = 2
#e = 6
#for i in range(0,1):
#  val = pow(n,e)
#  model.add(Conv2D(val,(3,3), padding = "same"))
#  model.add(Activation("softmax"))
#  model.add(MaxPooling2D())
#  e = e + 1

final layers

In [ ]:

#model.add(Flatten())
#model.add(Dense(128))
#model.add(Activation("softmax"))
#model.add(Dense(3,activation="softmax"))

In [ ]:
model.compile(Adam(lr = .0001), loss = 'categorical_crossentropy', metrics = ["accuracy"])

In [ ]:
model.fit(small_training_batch, steps_per_epoch= 3, epochs= 3)

Epoch 1/3
3/3 [==============================] - 0s 57ms/step - loss: 30.6277 - accuracy: 0.8889
Epoch 2/3
3/3 [==============================] - 0s 60ms/step - loss: 45.6530 - accuracy: 0.6667
Epoch 3/3
3/3 [==============================] - 0s 58ms/step - loss: 4.6480 - accuracy: 0.7778


In [ ]:
model.fit(training_Batches, steps_per_epoch= 20, validation_data = validation_Batches, validation_steps=12, epochs = 30) # callbacks=[eStopper])

20/20 [==============================] - 11s 560ms/step - loss: 1.0792 - accuracy: 0.5050 - val_loss: 1.0785 - val_accuracy: 0.5083


Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f225e57d5f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 535, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


Epoch 22/30
 1/20 [>.............................] - ETA: 4s - loss: 1.0906 - accuracy: 0.4118

KeyboardInterrupt: ignored

In [ ]:
#model.fit_generator(training_Batches, steps_per_epoch= 10, validation_data= validation_Batches, validation_steps=10, epochs = 10, verbose = 2)

**Predicting With Neural Network**

In [ ]:
testing_Batches.reset()

In [ ]:
predictions = model.predict(testing_Batches)

NameError: ignored

In [ ]:
predictions_proper = np.argmax(predictions,axis = 1)

In [ ]:
newPredfixed = np.argmax(newPred,axis = 1) + 1

In [ ]:
newPredfixed

array([3, 3, 2, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 2, 3, 2, 3, 3, 1, 3, 1,
       2, 1, 1, 1, 1, 1, 3, 3, 2, 1, 1, 3, 1, 1, 3, 1, 2, 3, 1, 2, 1, 1,
       3, 1, 3, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 3,
       2, 1, 1, 1, 1, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 3, 3, 1,
       3, 3, 2, 2, 2, 2, 3, 3, 1, 1, 3, 3, 1, 1, 3, 1, 1, 3, 1, 3, 1, 1,
       2, 1, 1, 1, 2, 1, 2, 3, 1, 3, 3, 3, 3, 1, 2, 2, 1, 1, 3, 2, 3, 3,
       2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 2, 3, 2, 3, 2, 3, 3, 3, 1, 3, 2, 3,
       3, 1, 2, 3, 3, 2, 1, 3, 3, 3, 3, 3, 1, 1, 2, 3, 1, 3, 3, 3, 1, 1,
       1, 3, 1, 2, 3, 1, 3, 2, 3, 1, 1, 1, 2, 3, 1, 2, 3, 3, 1, 2, 1, 3,
       1, 1, 3, 2, 2, 2, 1, 3, 2, 1, 2, 1, 3, 1, 2, 2, 2, 2, 3, 2, 2, 2,
       2, 2, 3, 3, 3, 2, 2, 3, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [ ]:
for i in range(0,predictions_proper.shape[0]):
  predictions_proper[i] = predictions_proper[i] + 1

In [ ]:
predictions_proper

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
testing_Batches.filenames

['all_classes/5309.jpeg',
 'all_classes/5310.jpeg',
 'all_classes/5311.jpeg',
 'all_classes/5312.jpeg',
 'all_classes/5313.jpeg',
 'all_classes/5314.jpeg',
 'all_classes/5315.jpeg',
 'all_classes/5316.jpeg',
 'all_classes/5317.jpeg',
 'all_classes/5318.jpeg',
 'all_classes/5319.jpeg',
 'all_classes/5320.jpeg',
 'all_classes/5321.jpeg',
 'all_classes/5322.jpeg',
 'all_classes/5323.jpeg',
 'all_classes/5324.jpeg',
 'all_classes/5325.jpeg',
 'all_classes/5326.jpeg',
 'all_classes/5327.jpeg',
 'all_classes/5328.jpeg',
 'all_classes/5329.jpeg',
 'all_classes/5330.jpeg',
 'all_classes/5331.jpeg',
 'all_classes/5332.jpeg',
 'all_classes/5333.jpeg',
 'all_classes/5334.jpeg',
 'all_classes/5335.jpeg',
 'all_classes/5336.jpeg',
 'all_classes/5337.jpeg',
 'all_classes/5338.jpeg',
 'all_classes/5339.jpeg',
 'all_classes/5340.jpeg',
 'all_classes/5341.jpeg',
 'all_classes/5342.jpeg',
 'all_classes/5343.jpeg',
 'all_classes/5344.jpeg',
 'all_classes/5345.jpeg',
 'all_classes/5346.jpeg',
 'all_classe

In [ ]:
sortedIDS = sorted(testingData.id)

In [ ]:
results = pd.DataFrame({"id": sortedIDS, "type": newPredfixed})
results = results.sort_values(by = ["id"])

In [ ]:
test = sorted(results.id)
test1 = results.id
print(test)

[5309, 5310, 5311, 5312, 5313, 5314, 5315, 5316, 5317, 5318, 5319, 5320, 5321, 5322, 5323, 5324, 5325, 5326, 5327, 5328, 5329, 5330, 5331, 5332, 5333, 5334, 5335, 5336, 5337, 5338, 5339, 5340, 5341, 5342, 5343, 5344, 5345, 5346, 5347, 5348, 5349, 5350, 5351, 5352, 5353, 5354, 5355, 5356, 5357, 5358, 5359, 5360, 5361, 5362, 5363, 5364, 5365, 5366, 5367, 5368, 5369, 5370, 5371, 5372, 5373, 5374, 5375, 5376, 5377, 5378, 5379, 5380, 5381, 5382, 5383, 5384, 5385, 5386, 5387, 5388, 5389, 5390, 5391, 5392, 5393, 5394, 5395, 5396, 5397, 5398, 5399, 5400, 5401, 5402, 5403, 5404, 5405, 5406, 5407, 5408, 5409, 5410, 5411, 5412, 5413, 5414, 5415, 5416, 5417, 5418, 5419, 5420, 5421, 5422, 5423, 5424, 5425, 5426, 5427, 5428, 5429, 5430, 5431, 5432, 5433, 5434, 5435, 5436, 5437, 5438, 5439, 5440, 5441, 5442, 5443, 5444, 5445, 5446, 5447, 5448, 5449, 5450, 5451, 5452, 5453, 5454, 5455, 5456, 5457, 5458, 5459, 5460, 5461, 5462, 5463, 5464, 5465, 5466, 5467, 5468, 5469, 5470, 5471, 5472, 5473, 5474, 547

In [ ]:
print(test1)

0      5309
1      5310
2      5311
3      5312
4      5313
       ... 
619    5928
620    5929
621    5930
622    5931
623    5932
Name: id, Length: 624, dtype: int64


In [ ]:
results.to_csv("codys_submission_as5.csv", index = False)

**Final Accuracy  Score: 91.185%**

Signing off  
-Cody